
- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [1]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import *

## Data Importing

In [2]:

file_name = 'AK_Juneau_0.json'  
data_dir = '../data/'
file_path = os.path.join(data_dir, file_name)

# Load  JSON
with open(file_path, 'r') as f:
    data_sample = json.load(f)


pprint(data_sample)

# load one file first to see what type of data you're dealing with and what attributes it has

first_sale = data_sample.get("data", {}).get("results", [])[0]

#branding_type = first_sale.get("branding", []).get("type")
branding_type = first_sale.get("branding", [])[0]
property_name = branding_type.get("name", {})
property_type = branding_type.get("type", {})
property_description = first_sale.get("description", {})
property_flags = first_sale.get("flags", {})
price_reduced_amount = first_sale.get("price_reduced_amount", {})
property_id = first_sale.get("property_id", {})
list_price = first_sale.get("list_price", {})
listing_id = first_sale.get("listing_id", {})
address_details = first_sale.get("location", {}).get("address", {})
#description = first_sale.get("description")

print(property_name, property_type, property_description, property_flags)


{'data': {'count': 8,
          'results': [{'branding': [{'name': 'EXP Realty LLC - Southeast '
                                             'Alaska',
                                     'photo': None,
                                     'type': 'Office'}],
                       'community': None,
                       'description': {'baths': 2,
                                       'baths_1qtr': None,
                                       'baths_3qtr': None,
                                       'baths_full': 2,
                                       'baths_half': None,
                                       'beds': 3,
                                       'garage': 1,
                                       'lot_sqft': 10454,
                                       'name': None,
                                       'sold_date': '2023-09-18',
                                       'sold_price': None,
                                       'sqft': 1821,
                      

In [3]:
# loop over all files and save the raw data to cvs for inspection

data_dir = '../data/'
all_sales = []

# Loop through and collect all property listings
for file in os.listdir(data_dir):
    if file.endswith('.json'):
        with open(os.path.join(data_dir, file), 'r') as f:
            try:
                data = json.load(f)
                results = data.get('data', {}).get('results', [])
                all_sales.extend(results)
            except Exception as e:
                print(f"Error loading {file}: {e}")

print(f"Total sales loaded: {len(all_sales)}")

# Flatten JSON structure
df = pd.json_normalize(all_sales)
df.to_csv('../data/housing_raw.csv', index=False)

Total sales loaded: 8159


In [4]:
# loop over all files and put them into a dataframe
 
all_records = []

# Loop through JSON files
for file in os.listdir(data_dir):
    if file.endswith(".json"):
        with open(os.path.join(data_dir, file), "r") as f:
            try:
                data = json.load(f)
                results = data.get("data", {}).get("results", [])
                
                for sale in results:
                    branding = sale.get("branding", [{}])[0]
                    full_location = sale.get("location", {}) or {}
                    address = full_location.get("address", {}) or {}
                    coordinates = address.get("coordinate", {})  or {}
                    county = full_location.get("county", {}) or {}

                    record = {
                        "property_name": branding.get("name"),
                        "property_type": branding.get("type"),
                        "baths": sale.get("description").get("baths"),
                        "baths_1qtr": sale.get("description").get("baths_1qtr"),
                        "baths_3qtr": sale.get("description").get("baths_3qtr"),
                        "baths_full": sale.get("description").get("baths_full"),
                        "baths_half": sale.get("description").get("baths_half"),
                        "beds": sale.get("description").get("beds"),
                        "garage": sale.get("description").get("garage"),
                        "lot_sqft": sale.get("description").get("lot_sqft"),
                        "name": sale.get("description").get("name"),
                        "sold_date": sale.get("description").get("sold_date"),
                        "sold_price": sale.get("description").get("sold_price"),
                        "sqft": sale.get("description").get("sqft"),
                        "stories": sale.get("description").get("stories"),
                        "sub_type": sale.get("description").get("sub_type"),
                        "type": sale.get("description").get("type"),
                        "year_built": sale.get("description").get("year_built"),
                        "is_foreclosure": sale.get("flags").get("is_foreclosure"),
                        "is_price_reduced": sale.get("flags").get("is_price_reduced"),
                        "is_new_listing": sale.get("flags").get("is_new_listing"),
                        "is_new_construction": sale.get("flags").get("is_new_construction"),
                        "price_reduced_amount": sale.get("price_reduced_amount"),
                        "property_id": sale.get("property_id"),
                        "list_price": sale.get("list_price"),
                        "listing_id": sale.get("listing_id"),
                        "list_date": sale.get("list_date"),
                        "street_address": address.get("line"),
                        "city": address.get("city"),
                        "state": address.get("state_code"),
                        "country": address.get("country"),
                        "county_name": county.get("name"),
                        "postal_code": address.get("postal_code"),
                        "latitude": coordinates.get("lat"),
                        "longitude": coordinates.get("lon"),
                    }

                    all_records.append(record)

            except Exception as e:
                print(f"Error loading {file}: {e}")

# Convert to DataFrame
df = pd.DataFrame(all_records)

# Save to CSV
os.makedirs("../data/processed", exist_ok=True)
df.to_csv("../data/processed/property_listings_flat.csv", index=False)

print("✅ Finished! Saved to data/processed/property_listings_flat.csv")


✅ Finished! Saved to data/processed/property_listings_flat.csv


In [5]:
df

,property_name,property_type,baths,baths_1qtr,baths_3qtr,baths_full,baths_half,beds,garage,lot_sqft,...,listing_id,list_date,street_address,city,state,country,county_name,postal_code,latitude,longitude
0,EXP Realty LLC - Southeast Alaska,Office,2.0,None,NaN,2.0,NaN,3.0,1.0,10454.0,...,2957241843,2023-06-29T21:16:25.000000Z,9453 Herbert Pl,Juneau,AK,None,Juneau,99801,58.363950,-134.593720
1,None,Office,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,8477 Thunder Mountain Rd,Juneau,AK,None,Juneau,99801,NaN,NaN
2,None,Office,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,4515 Glacier Hwy,Juneau,AK,None,Juneau,99801,NaN,NaN
3,None,Office,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,17850 Point Stephens Rd,Juneau,AK,None,Juneau,99801,NaN,NaN
4,None,Office,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,9951 Stephen Richards Memorial Dr,Juneau,AK,None,Juneau,99801,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8154,Joe R. Pyle Complete Auction And Realty Service,Office,1.0,None,NaN,1.0,NaN,3.0,1.0,4792.0,...,2954257214,2023-04-05T19:28:04Z,1008 Oakmont Rd,Charleston,WV,None,Kanawha,25314,38.341576,-81.644994
8155,Joe R. Pyle Complete Auction And Realty Service,Office,1.0,None,NaN,1.0,NaN,3.0,NaN,7841.0,...,2956178043,2023-05-28T02:25:28Z,1041 Temple St,Charleston,WV,None,Kanawha,25387,38.377371,-81.661662
8156,Joe R. Pyle Complete Auction And Realty Service,Office,1.0,None,NaN,1.0,NaN,3.0,NaN,65340.0,...,2951731627,2023-01-24T18:27:46Z,238 Oakwood Rd,Charleston,WV,None,Kanawha,25314,38.338617,-81.659885
8157,Joe R. Pyle Complete Auction And Realty Service,Office,0.0,None,NaN,NaN,NaN,0.0,NaN,NaN,...,2953862293,2023-03-24T12:27:42Z,408 Lee St W,Charleston,WV,None,Kanawha,25302,38.363038,-81.644214


## Data Cleaning and Wrangling

At this point, ensure that you have all sales in a dataframe.
- Take a quick look at your data (i.e. `.info()`, `.describe()`) - what do you see?
- Is each cell one value, or do some cells have lists?
- What are the data types of each column?
- Some sales may not actually include the sale price (target).  These rows should be dropped.
- There are a lot of NA/None values.  Should these be dropped or replaced with something?
    - You can drop rows or use various methods to fills NA's - use your best judgement for each column 
    - i.e. for some columns (like Garage), NA probably just means no Garage, so 0
- Drop columns that aren't needed
    - Don't keep the list price because it will be too close to the sale price. Assume we want to predict the price of houses not yet listed

In [6]:
# load and concatenate data here

df = pd.read_csv(r"../data/processed/property_listings_flat.csv")
df.head()
df.describe()
df.info()
df.columns



# drop or replace values as necessary

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8159 entries, 0 to 8158
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   property_name         7725 non-null   object 
 1   property_type         8159 non-null   object 
 2   baths                 7980 non-null   float64
 3   baths_1qtr            0 non-null      float64
 4   baths_3qtr            566 non-null    float64
 5   baths_full            7311 non-null   float64
 6   baths_half            2281 non-null   float64
 7   beds                  7504 non-null   float64
 8   garage                4448 non-null   float64
 9   lot_sqft              6991 non-null   float64
 10  name                  0 non-null      float64
 11  sold_date             8159 non-null   object 
 12  sold_price            6716 non-null   float64
 13  sqft                  7323 non-null   float64
 14  stories               6260 non-null   float64
 15  sub_type             

Index(['property_name', 'property_type', 'baths', 'baths_1qtr', 'baths_3qtr',
       'baths_full', 'baths_half', 'beds', 'garage', 'lot_sqft', 'name',
       'sold_date', 'sold_price', 'sqft', 'stories', 'sub_type', 'type',
       'year_built', 'is_foreclosure', 'is_price_reduced', 'is_new_listing',
       'is_new_construction', 'price_reduced_amount', 'property_id',
       'list_price', 'listing_id', 'list_date', 'street_address', 'city',
       'state', 'country', 'county_name', 'postal_code', 'latitude',
       'longitude'],
      dtype='object')

In [7]:
# calls a function that deletes a column if all values are null

df_cleaned = drop_all_null_columns(df)
df_cleaned

,property_name,property_type,baths,baths_3qtr,baths_full,baths_half,beds,garage,lot_sqft,sold_date,...,list_price,listing_id,list_date,street_address,city,state,county_name,postal_code,latitude,longitude
0,EXP Realty LLC - Southeast Alaska,Office,2.0,NaN,2.0,NaN,3.0,1.0,10454.0,2023-09-18,...,554950.0,2.957242e+09,2023-06-29T21:16:25.000000Z,9453 Herbert Pl,Juneau,AK,Juneau,99801,58.363950,-134.593720
1,NaN,Office,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-22,...,NaN,NaN,NaN,8477 Thunder Mountain Rd,Juneau,AK,Juneau,99801,NaN,NaN
2,NaN,Office,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-22,...,NaN,NaN,NaN,4515 Glacier Hwy,Juneau,AK,Juneau,99801,NaN,NaN
3,NaN,Office,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-21,...,NaN,NaN,NaN,17850 Point Stephens Rd,Juneau,AK,Juneau,99801,NaN,NaN
4,NaN,Office,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-21,...,NaN,NaN,NaN,9951 Stephen Richards Memorial Dr,Juneau,AK,Juneau,99801,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8154,Joe R. Pyle Complete Auction And Realty Service,Office,1.0,NaN,1.0,NaN,3.0,1.0,4792.0,2023-07-27,...,98000.0,2.954257e+09,2023-04-05T19:28:04Z,1008 Oakmont Rd,Charleston,WV,Kanawha,25314,38.341576,-81.644994
8155,Joe R. Pyle Complete Auction And Realty Service,Office,1.0,NaN,1.0,NaN,3.0,NaN,7841.0,2023-07-27,...,49999.0,2.956178e+09,2023-05-28T02:25:28Z,1041 Temple St,Charleston,WV,Kanawha,25387,38.377371,-81.661662
8156,Joe R. Pyle Complete Auction And Realty Service,Office,1.0,NaN,1.0,NaN,3.0,NaN,65340.0,2023-07-24,...,133000.0,2.951732e+09,2023-01-24T18:27:46Z,238 Oakwood Rd,Charleston,WV,Kanawha,25314,38.338617,-81.659885
8157,Joe R. Pyle Complete Auction And Realty Service,Office,0.0,NaN,NaN,NaN,0.0,NaN,NaN,2023-07-24,...,75000.0,2.953862e+09,2023-03-24T12:27:42Z,408 Lee St W,Charleston,WV,Kanawha,25302,38.363038,-81.644214


In [8]:
df_modified = fill_missing_with_mean(df, columns=["beds", "baths", "garage"])
df_modified

,property_name,property_type,baths,baths_1qtr,baths_3qtr,baths_full,baths_half,beds,garage,lot_sqft,...,listing_id,list_date,street_address,city,state,country,county_name,postal_code,latitude,longitude
0,EXP Realty LLC - Southeast Alaska,Office,2.000000,NaN,NaN,2.0,NaN,3.000000,1.000000,10454.0,...,2.957242e+09,2023-06-29T21:16:25.000000Z,9453 Herbert Pl,Juneau,AK,NaN,Juneau,99801,58.363950,-134.593720
1,NaN,Office,2.131203,NaN,NaN,NaN,NaN,3.208289,1.926709,NaN,...,NaN,NaN,8477 Thunder Mountain Rd,Juneau,AK,NaN,Juneau,99801,NaN,NaN
2,NaN,Office,2.131203,NaN,NaN,NaN,NaN,3.208289,1.926709,NaN,...,NaN,NaN,4515 Glacier Hwy,Juneau,AK,NaN,Juneau,99801,NaN,NaN
3,NaN,Office,2.131203,NaN,NaN,NaN,NaN,3.208289,1.926709,NaN,...,NaN,NaN,17850 Point Stephens Rd,Juneau,AK,NaN,Juneau,99801,NaN,NaN
4,NaN,Office,2.131203,NaN,NaN,NaN,NaN,3.208289,1.926709,NaN,...,NaN,NaN,9951 Stephen Richards Memorial Dr,Juneau,AK,NaN,Juneau,99801,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8154,Joe R. Pyle Complete Auction And Realty Service,Office,1.000000,NaN,NaN,1.0,NaN,3.000000,1.000000,4792.0,...,2.954257e+09,2023-04-05T19:28:04Z,1008 Oakmont Rd,Charleston,WV,NaN,Kanawha,25314,38.341576,-81.644994
8155,Joe R. Pyle Complete Auction And Realty Service,Office,1.000000,NaN,NaN,1.0,NaN,3.000000,1.926709,7841.0,...,2.956178e+09,2023-05-28T02:25:28Z,1041 Temple St,Charleston,WV,NaN,Kanawha,25387,38.377371,-81.661662
8156,Joe R. Pyle Complete Auction And Realty Service,Office,1.000000,NaN,NaN,1.0,NaN,3.000000,1.926709,65340.0,...,2.951732e+09,2023-01-24T18:27:46Z,238 Oakwood Rd,Charleston,WV,NaN,Kanawha,25314,38.338617,-81.659885
8157,Joe R. Pyle Complete Auction And Realty Service,Office,0.000000,NaN,NaN,NaN,NaN,0.000000,1.926709,NaN,...,2.953862e+09,2023-03-24T12:27:42Z,408 Lee St W,Charleston,WV,NaN,Kanawha,25302,38.363038,-81.644214


### Dealing with Tags

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [9]:
# OHE categorical variables/ tags here
# tags will have to be done manually

### Dealing with Cities

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- What we can do is use our training data to encode the mean sale price by city as a feature (a.k.a. Target Encoding)
    - We can do this as long as we ONLY use the training data - we're using the available data to give us a 'starting guess' of the price for each city, without needing to encode city explicitly
- If you replace cities or states with numerical values (like the mean price), make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Note that you *may* have cities in the test set that are not in the training set. You don't want these to be NA, so maybe you can fill them with the overall mean

In [10]:
# perform train test split here
# do something with state and city

## Extra Data - STRETCH

> This doesn't need to be part of your Minimum Viable Product (MVP). We recommend you write a functional, basic pipeline first, then circle back and join new data if you have time

> If you do this, try to write your downstream steps in a way it will still work on a dataframe with different features!

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [11]:
# import, join and preprocess new data here

## EDA/ Visualization

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.    
    - Consider transforming very skewed variables
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
    - You may have too many features to do this, in which case you can simply compute the most correlated feature-pairs and list them
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [12]:
# perform EDA here

## Scaling and Finishing Up

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized